In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import csv
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress

# Import API key
#from api_keys import geoapify_key


In [4]:
# Specify the path to your CSV file
csvpath = 'Resources/reduced.csv'

# Read the CSV file into a DataFrame
data = pd.read_csv(csvpath)
data_df = pd.DataFrame(data)
# Display the DataFrame
data_df


,Unnamed: 0,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,357091,Texas,Female,Very good,0.0,30.0,Within past year (anytime less than 12 months ...,Yes,6.0,None of them,...,1.65,58.97,21.63,No,No,No,No,"Yes, received tetanus shot but not sure what type",No,Yes
1,199859,Minnesota,Female,Good,6.0,25.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,...,1.55,56.70,23.62,Yes,Yes,No,Yes,"Yes, received tetanus shot but not sure what type",Yes,No
2,347964,Texas,Female,Good,10.0,20.0,Within past year (anytime less than 12 months ...,Yes,12.0,"6 or more, but not all",...,1.60,61.23,23.91,Yes,No,No,Yes,"Yes, received Tdap",No,No
3,407343,Washington,Female,Fair,0.0,19.0,Within past year (anytime less than 12 months ...,No,8.0,None of them,...,1.57,NaN,NaN,Yes,No,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,Tested positive using home test without a heal...
4,211281,Mississippi,Male,Very good,4.0,0.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,...,1.85,95.25,27.71,Yes,No,Yes,No,"Yes, received tetanus shot, but not Tdap",No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222561,66670,Florida,Male,Good,0.0,7.0,5 or more years ago,Yes,8.0,1 to 5,...,1.91,79.38,21.87,Yes,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,No
222562,259454,New York,Male,Poor,30.0,30.0,Within past year (anytime less than 12 months ...,No,9.0,"6 or more, but not all",...,1.73,87.54,29.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
222563,8440,Alaska,Female,Very good,10.0,10.0,Within past year (anytime less than 12 months ...,No,8.0,None of them,...,1.50,61.23,27.27,Yes,No,No,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
222564,132209,Kansas,Male,Very good,0.0,15.0,Within past year (anytime less than 12 months ...,Yes,6.0,None of them,...,1.83,124.74,37.30,Yes,No,No,No,"Yes, received tetanus shot but not sure what type",No,No


In [11]:
print(data_df.columns)

Index(['State', 'Sex', 'GeneralHealth', 'PhysicalHealthDays',
       'MentalHealthDays', 'LastCheckupTime', 'PhysicalActivities',
       'SleepHours', 'RemovedTeeth', 'HadHeartAttack', 'HadAngina',
       'HadStroke', 'HadAsthma', 'HadSkinCancer', 'HadCOPD',
       'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis',
       'HadDiabetes', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty',
       'DifficultyConcentrating', 'DifficultyWalking',
       'DifficultyDressingBathing', 'DifficultyErrands', 'SmokerStatus',
       'ECigaretteUsage', 'ChestScan', 'RaceEthnicityCategory', 'AgeCategory',
       'HeightInMeters', 'WeightInKilograms', 'BMI', 'AlcoholDrinkers',
       'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver', 'TetanusLast10Tdap',
       'HighRiskLastYear', 'CovidPos'],
      dtype='object')


In [13]:
#Remove unneeded columns
cleaned_data_df=data_df[['State', 'Sex','GeneralHealth', 'PhysicalHealthDays', 'MentalHealthDays',	'PhysicalActivities', 
                         'SleepHours','HadDepressiveDisorder','BMI', 'AlcoholDrinkers', 'ECigaretteUsage','SmokerStatus']]
cleaned_data_df

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,PhysicalActivities,SleepHours,HadDepressiveDisorder,BMI,AlcoholDrinkers,ECigaretteUsage,SmokerStatus
0,Alabama,Female,Very good,0.0,0.0,No,8.0,No,NaN,No,Not at all (right now),Never smoked
1,Alabama,Female,Excellent,0.0,0.0,No,6.0,No,26.57,No,Never used e-cigarettes in my entire life,Never smoked
2,Alabama,Female,Very good,2.0,3.0,Yes,5.0,No,25.61,No,Never used e-cigarettes in my entire life,Never smoked
3,Alabama,Female,Excellent,0.0,0.0,Yes,7.0,No,23.30,No,Never used e-cigarettes in my entire life,Current smoker - now smokes some days
4,Alabama,Female,Fair,2.0,0.0,Yes,9.0,No,21.77,Yes,Never used e-cigarettes in my entire life,Never smoked
...,...,...,...,...,...,...,...,...,...,...,...,...
445127,Virgin Islands,Female,Good,0.0,3.0,Yes,6.0,Yes,25.63,NaN,Never used e-cigarettes in my entire life,Never smoked
445128,Virgin Islands,Female,Excellent,2.0,2.0,Yes,7.0,No,28.66,No,Never used e-cigarettes in my entire life,Never smoked
445129,Virgin Islands,Female,Poor,30.0,30.0,No,5.0,No,17.23,NaN,Use them some days,Current smoker - now smokes every day
445130,Virgin Islands,Male,Very good,0.0,0.0,No,5.0,No,32.55,No,Never used e-cigarettes in my entire life,Never smoked


In [14]:
#Check for columns with 'Nan' values
non_nan_counts = cleaned_data_df.count()
print(non_nan_counts)

State                    445132
Sex                      445132
GeneralHealth            443934
PhysicalHealthDays       434205
MentalHealthDays         436065
PhysicalActivities       444039
SleepHours               439679
HadDepressiveDisorder    442320
BMI                      396326
AlcoholDrinkers          398558
ECigaretteUsage          409472
SmokerStatus             409670
dtype: int64


In [16]:
#Remove 'Nan' values from data
cleaned_data2_df=cleaned_data_df.dropna()
cleaned_data2_df

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,PhysicalActivities,SleepHours,HadDepressiveDisorder,BMI,AlcoholDrinkers,ECigaretteUsage,SmokerStatus
1,Alabama,Female,Excellent,0.0,0.0,No,6.0,No,26.57,No,Never used e-cigarettes in my entire life,Never smoked
2,Alabama,Female,Very good,2.0,3.0,Yes,5.0,No,25.61,No,Never used e-cigarettes in my entire life,Never smoked
3,Alabama,Female,Excellent,0.0,0.0,Yes,7.0,No,23.30,No,Never used e-cigarettes in my entire life,Current smoker - now smokes some days
4,Alabama,Female,Fair,2.0,0.0,Yes,9.0,No,21.77,Yes,Never used e-cigarettes in my entire life,Never smoked
5,Alabama,Male,Poor,1.0,0.0,No,7.0,No,26.08,No,Never used e-cigarettes in my entire life,Never smoked
...,...,...,...,...,...,...,...,...,...,...,...,...
445124,Virgin Islands,Male,Good,0.0,15.0,Yes,7.0,No,29.86,Yes,Never used e-cigarettes in my entire life,Never smoked
445126,Virgin Islands,Male,Good,0.0,0.0,Yes,8.0,No,31.19,Yes,Never used e-cigarettes in my entire life,Never smoked
445128,Virgin Islands,Female,Excellent,2.0,2.0,Yes,7.0,No,28.66,No,Never used e-cigarettes in my entire life,Never smoked
445130,Virgin Islands,Male,Very good,0.0,0.0,No,5.0,No,32.55,No,Never used e-cigarettes in my entire life,Never smoked


In [17]:
#Make sure 'Nan' values are removed
non_nan_counts = cleaned_data2_df.count()
print(non_nan_counts)

State                    351259
Sex                      351259
GeneralHealth            351259
PhysicalHealthDays       351259
MentalHealthDays         351259
PhysicalActivities       351259
SleepHours               351259
HadDepressiveDisorder    351259
BMI                      351259
AlcoholDrinkers          351259
ECigaretteUsage          351259
SmokerStatus             351259
dtype: int64


In [19]:
Number_of_states=cleaned_data2_df['State'].unique()
print(Number_of_states)

['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'District of Columbia' 'Florida' 'Georgia'
 'Hawaii' 'Idaho' 'Illinois' 'Indiana' 'Iowa' 'Kansas' 'Kentucky'
 'Louisiana' 'Maine' 'Maryland' 'Massachusetts' 'Michigan' 'Minnesota'
 'Mississippi' 'Missouri' 'Montana' 'Nebraska' 'Nevada' 'New Hampshire'
 'New Jersey' 'New Mexico' 'New York' 'North Carolina' 'North Dakota'
 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania' 'Rhode Island' 'South Carolina'
 'South Dakota' 'Tennessee' 'Texas' 'Utah' 'Vermont' 'Virginia'
 'Washington' 'West Virginia' 'Wisconsin' 'Wyoming' 'Guam' 'Puerto Rico'
 'Virgin Islands']


In [24]:
#Remove Guam, Puerto Rico and VI
values_to_remove=['Guam','Puerto Rico','Virgin Islands','District of Columbia']
cleaned_data3_df=cleaned_data2_df[~cleaned_data2_df['State'].isin(values_to_remove)]
cleaned_data3_df

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,PhysicalActivities,SleepHours,HadDepressiveDisorder,BMI,AlcoholDrinkers,ECigaretteUsage,SmokerStatus
1,Alabama,Female,Excellent,0.0,0.0,No,6.0,No,26.57,No,Never used e-cigarettes in my entire life,Never smoked
2,Alabama,Female,Very good,2.0,3.0,Yes,5.0,No,25.61,No,Never used e-cigarettes in my entire life,Never smoked
3,Alabama,Female,Excellent,0.0,0.0,Yes,7.0,No,23.30,No,Never used e-cigarettes in my entire life,Current smoker - now smokes some days
4,Alabama,Female,Fair,2.0,0.0,Yes,9.0,No,21.77,Yes,Never used e-cigarettes in my entire life,Never smoked
5,Alabama,Male,Poor,1.0,0.0,No,7.0,No,26.08,No,Never used e-cigarettes in my entire life,Never smoked
...,...,...,...,...,...,...,...,...,...,...,...,...
435818,Wyoming,Male,Excellent,0.0,0.0,Yes,6.0,No,22.96,Yes,Never used e-cigarettes in my entire life,Never smoked
435819,Wyoming,Female,Excellent,0.0,0.0,Yes,8.0,No,22.86,Yes,Never used e-cigarettes in my entire life,Current smoker - now smokes some days
435820,Wyoming,Male,Fair,2.0,2.0,No,6.0,Yes,26.83,No,Not at all (right now),Current smoker - now smokes every day
435823,Wyoming,Male,Good,7.0,0.0,Yes,8.0,No,29.56,Yes,Never used e-cigarettes in my entire life,Former smoker


In [26]:
#Verify the rows were removed
Number_of_states=cleaned_data3_df['State'].nunique()
print(Number_of_states)

50


In [27]:
#Randomly sample a portion of the data to make it easier to work with (optional); frac value is a percentage 
Random_df=cleaned_data3_df.sample(frac=0.1, random_state=42)
Random_df

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,PhysicalActivities,SleepHours,HadDepressiveDisorder,BMI,AlcoholDrinkers,ECigaretteUsage,SmokerStatus
349002,Texas,Female,Good,0.0,0.0,Yes,6.0,No,30.23,Yes,Never used e-cigarettes in my entire life,Never smoked
256132,New Mexico,Female,Good,0.0,4.0,No,9.0,No,24.02,Yes,Never used e-cigarettes in my entire life,Former smoker
309602,Oregon,Female,Good,0.0,0.0,Yes,6.0,No,36.22,No,Never used e-cigarettes in my entire life,Never smoked
145688,Maine,Female,Good,0.0,0.0,Yes,6.0,No,22.46,No,Never used e-cigarettes in my entire life,Former smoker
15092,Arizona,Male,Good,0.0,25.0,Yes,7.0,No,30.87,Yes,Not at all (right now),Former smoker
...,...,...,...,...,...,...,...,...,...,...,...,...
300381,Ohio,Female,Good,2.0,10.0,Yes,7.0,No,36.61,No,Not at all (right now),Current smoker - now smokes every day
74658,Florida,Female,Good,0.0,2.0,Yes,5.0,No,20.60,Yes,Never used e-cigarettes in my entire life,Never smoked
3243,Alabama,Female,Very good,0.0,4.0,No,8.0,No,27.46,Yes,Not at all (right now),Never smoked
217436,Missouri,Female,Excellent,0.0,0.0,No,4.0,No,24.21,Yes,Never used e-cigarettes in my entire life,Never smoked
